In [251]:
from peewee import *

In [252]:
sqlite_db = SqliteDatabase('tests.db')

In [253]:
class BaseModel(Model):
    class Meta:
        database = sqlite_db

In [254]:
class Test(BaseModel):
    year = IntegerField()

In [255]:
class Question(BaseModel):
    number = IntegerField()
    description = CharField()
    type = CharField()
    test_id = ForeignKeyField(Test, backref='questions')

In [256]:
class Answer(BaseModel):
    marker = CharField()
    description = CharField()
    correct = BooleanField(default=False)
    question_id = ForeignKeyField(Question, backref='answers')

In [257]:
sqlite_db.create_tables([Test, Question, Answer])

In [258]:
from bs4 import BeautifulSoup

In [259]:
!ls

2002                  2009                  2016
2003                  2010                  2017
2004                  2011                  2018
2005                  2012                  2019
2006                  2013                  generate_sqlite.ipynb
2007                  2014                  tests.db
2008                  2015


In [260]:
html = open('2019/test.html').read()

In [261]:
import re
import base64

In [262]:
for image in re.findall("\d*.png", html):
    html = html.replace(
        image, 
        'data:image/png;base64,' + base64.b64encode(open('2019/images/'+image, 'rb').read()).decode('utf-8')
    )

In [263]:
soup = BeautifulSoup(html, 'html.parser')

In [264]:
def get_type(index):
    if(index < 20):
        return 'MAT'
    elif(index < 50):
        return 'FUN'
    else:
        return 'TEC'

In [265]:
t = Test(year = int(soup.find('year').text))
t.save()
for index, question in enumerate(soup.find_all('question')):
    q = Question(
        number=int(question.find('number').text),
        description=question.find('description').p,
        type=get_type(index),
        test_id=t.id
    )
    q.save()
    for marker in ['a', 'b', 'c', 'd', 'e']:
        a = Answer(
            marker=marker,
            description=question.find(marker).text,
            correct='correct' in question.find(marker).attrs,
            question_id=q.id
        )
        a.save()